# Speed Fit workflow

This notebook uses `speed_fit/` drivers to run a faster Method A pipeline (iterative channel fill + batch helpers).

In [ ]:
from pathlib import Path
import sys

cwd = Path.cwd().resolve()
repo_root = cwd if (cwd / 'python_code').exists() else cwd.parent
assert (repo_root / 'python_code').exists(), f'Could not find repo root from {cwd}'
python_code_dir = repo_root / 'python_code'
speed_fit_dir = repo_root / 'speed_fit'
for p in (python_code_dir, speed_fit_dir):
    if str(p) not in sys.path:
        sys.path.insert(0, str(p))
data_csv = repo_root / 'data' / 'work.csv'
print('repo_root:', repo_root)
print('data_csv exists:', data_csv.exists())


In [ ]:
missing = []
for pkg in ('numpy', 'iminuit'):
    try:
        __import__(pkg)
    except Exception:
        missing.append(pkg)
if missing:
    raise RuntimeError('Missing dependencies: ' + ', '.join(missing))
from fast_fit_driver import fit_noE_fast, fit_noE_batch
print('Dependencies loaded.')


In [ ]:
# Fast single-run sanity (one simulation only, no minimization)
methodA, best, minuit = fit_noE_fast(str(data_csv), quick_sanity=True, verbose=False)
print('quick_sanity mode:', minuit is None)
print('best (seed)[:6] =', best[:6])


In [ ]:
# Batch processing API (serial here; set workers>1 for multiprocess)
results = fit_noE_batch([str(data_csv), str(data_csv)], workers=1, quick_sanity=True)
[(r.csv_path, r.quick_sanity, r.fval) for r in results]
